In [ ]:
# pip install  pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 4.2 MB/s 
     |████████████████████████████████| 9.8 MB 44.6 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
import pandas as pd
import numpy as np
import warnings
from sklearn import metrics as mt
from pmdarima.arima import auto_arima

In [ ]:
y_train = pd.read_csv('y_train_complete.csv', index_col=['date'], parse_dates=['date'])
y_test = pd.read_csv('y_test.csv', index_col=['date'], parse_dates=['date'])

In [ ]:
y_test.index.name = 'Date'
y_test.rename(columns={"reference_evapotranspiration": "REFERENCE_EVAPOTRANSPIRATION"});

In [ ]:
reference_evapotranspiration = pd.concat([y_train, y_test])

In [ ]:
sarima_model = auto_arima(reference_evapotranspiration.reference_evapotranspiration,
                          start_p=1, start_q=1, max_p=10, max_q=10, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

sarima_model.summary()

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=inf, Time=3.18 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=-1809.343, Time=0.20 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=-1872.898, Time=1.51 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=2.95 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=-1823.336, Time=0.23 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=-1901.077, Time=1.67 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=6.69 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=inf, Time=2.98 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=-1888.021, Time=0.47 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=-1911.429, Time=3.57 sec
 ARIMA(2,1,0)(1,1,0)[12]             : AIC=-1884.508, Time=1.46 sec
 ARIMA(2,1,0)(2,1,1)[12]             : AIC=inf, Time=6.67 sec
 ARIMA(2,1,0)(1,1,1)[12]             : AIC=inf, Time=3.84 sec
 ARIMA(3,1,0)(2,1,0)[12]             : AIC=-1916.004, Time=2.22 sec
 ARIMA(3,1,0)(1,1,0)[12]             : AI

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  334
Model:             SARIMAX(5, 1, 0)x(2, 1, [1], 12)   Log Likelihood                 997.229
Date:                              Sat, 04 Jun 2022   AIC                          -1976.458
Time:                                      05:54:00   BIC                          -1942.515
Sample:                                           0   HQIC                         -1962.905
                                              - 334                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.2979      0.056     -5.368      0.000      -0.407      -0.189
ar.L2         -0.2542      0.066     -3.876      0.000      -0.383      -0.126
ar.L3         -0.1973      0.062     -3.172      0.002      -0.319      -0.075
ar.L4         -0.1286      0.067     -1.927      0.054      -0.259       0.002
ar.L5         -0.0950      0.065     -1.460      0.144      -0.223       0.033
ar.S.L12       0.0283      0.070      0.402      0.688      -0.110       0.166
ar.S.L24      -0.1134      0.077     -1.472      0.141      -0.264       0.038
ma.S.L12      -0.8598      0.063    -13.603      0.000      -0.984      -0.736
sigma2         0.0001   9.87e-06     12.005      0.000    9.92e-05       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 7.91
Prob(Q):                              0.96   Prob(JB):                         0.02
Heteroskedasticity (H):               0.98   Skew:                            -0.21
Prob(H) (two-sided):                  0.91   Kurtosis:                         3.64
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
sarima_model.fit(y_train.reference_evapotranspiration)

ARIMA(order=(5, 1, 0), scoring_args={}, seasonal_order=(2, 1, 1, 12),
      suppress_warnings=True, with_intercept=False)

In [ ]:
y_pred = sarima_model.predict(n_periods=len(y_test))

print('RMSE = {}\nMAE  = {}\n'.format(
    np.sqrt(mt.mean_squared_error(y_test.reference_evapotranspiration, y_pred)), 
    mt.mean_absolute_error(y_test.reference_evapotranspiration, y_pred)))

RMSE = 0.025276322446217858
MAE  = 0.023053239228703216

